In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

import os
import re
import numpy as np
import pandas as pd
import collections

In [2]:
x_name = os.path.join(os.getcwd(), '데이터SET', f"{'[Track1_데이터3] samp_cst_feat'}.csv")
df_x = pd.read_csv(x_name, index_col=0)

y_name = os.path.join(os.getcwd(), '데이터SET', f"{'[Track1_데이터2] samp_train'}.csv")
df_y = pd.read_csv(y_name, index_col=0)

df = pd.merge(df_x, df_y, on='cst_id_di')

In [3]:
df.MRC_ID_DI[df.MRC_ID_DI > 0 ] = 1

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [4]:
nc_name = os.path.join(os.getcwd(), '데이터SET', f"{'[Track1_데이터4] variable_dtype'}.xlsx")
nc = pd.read_excel(nc_name, index_col=0)

In [ ]:
df

In [ ]:
%matplotlib inline
plt.figure(figsize = (227, 227))
sns.heatmap(df.corr(), annot = True, fmt = '.2f', square = True)
plt.savefig('corr13.png')
plt.show()

In [ ]:
pd.set_option('display.max_columns', None)
df.describe([.01, .10, .20, .30, .40, .50, .60, .70, .80, .90, .95, .99])

In [ ]:
for i in df.columns[0:]:
    if i == 'MRC_ID_DI' or nc.loc[i, 'dType'] == 'categorical':
        ax = sns.countplot(df[i], palette=['#432371',"#FAAE7B"])
        plt.show()
    else:
        ax = sns.boxplot(x=i, data=df, color = 'yellow')
        ax.set(xlim=(-1, 1))
        plt.show()

In [ ]:
#https://www.pluralsight.com/guides/cleaning-up-data-from-outliers
#https://towardsdatascience.com/ways-to-detect-and-remove-the-outliers-404d16608dba
Q1 = df.quantile(0.05)
Q3 = df.quantile(0.95)
IQR = Q3 - Q1
df_out = df[~((df < (Q1 - 1.5 * IQR)) |(df > (Q3 + 1.5 * IQR))).any(axis=1)]
print(df_out.shape)

In [ ]:
#https://www.kite.com/python/answers/how-to-remove-outliers-from-a-pandas-dataframe-in-python
from scipy import stats
import numpy as np

z_scores = stats.zscore(df)

abs_z_scores = np.abs(z_scores)
filtered_entries = (abs_z_scores < 5).all(axis=1)
df = df[filtered_entries]
print(df.shape)

In [5]:
"""
drop_columns = list()
for i in df.columns[0:-1]:
    if nc.loc[i, 'dType'] == 'numerical':
        if df[i].quantile(0.75) - df[i].quantile(0.25) > 0.5:
            print(i)
            drop_columns.append(i)
    else:
        if len(df[df[i] == 1]) > len(df)*0.8 or len(df[df[i] == 0]) > len(df)*0.8:
            drop_columns.append(i)

df = df.drop(columns = drop_columns, axis=1)
"""
#https://www.pluralsight.com/guides/cleaning-up-data-from-outliers
for i in df.columns[0:-1]:
    if nc.loc[i, 'dType'] == 'numerical':
        d_90 = df[i].quantile(0.99)
        d_10 = df[i].quantile(0.01)
        d_50 = df[i].quantile(0.50)
        df[i] = np.where(df[i] > d_90, d_90, df[i])
        df[i] = np.where(df[i] < d_10, d_10, df[i])
print(df.shape)

(10124, 227)


In [6]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
import random

physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)

seed_value= 0
os.environ['PYTHONHASHSEED']=str(seed_value)
random.seed(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)

In [7]:
import sys
ipython = get_ipython()

def hide_traceback(exc_tuple=None, filename=None, tb_offset=None,
                      exception_only=False, running_compiled_code=False):
       etype, value, tb = sys.exc_info()
       return ipython._showtraceback(etype, value, ipython.InteractiveTB.get_exception_only(etype, value))

ipython.showtraceback = hide_traceback

In [20]:
df_0 = df[df['MRC_ID_DI'] == 0].sample(frac=1)
df_1 = df[df['MRC_ID_DI'] == 1].sample(frac=1)

sample_size = len(df_0) if len(df_0) < len(df_1) else len(df_1)

df_f = pd.concat([df_0.head(sample_size), df_1.head(sample_size)]).sample(frac=1)
df_t = pd.concat([df_0.tail(sample_size), df_1.head(sample_size)]).sample(frac=1)

In [21]:
#X = df_f.drop('MRC_ID_DI', axis=1)
X = df_f.drop(columns = ['MRC_ID_DI', 'VAR021', 'VAR046'], axis=1)


#X = df_f.drop(columns = ['MRC_ID_DI', 'VAR006', 'VAR012', 'VAR021', 'VAR034', 'VAR046', 'VAR047', 'VAR056', 'VAR069', 'VAR075', 'VAR089', 
#                        'VAR093', 'VAR106', 'VAR116', 'VAR123', 'VAR133', 'VAR134', 'VAR147', 'VAR184', 'VAR203', 'VAR214', 'VAR215', 'VAR218', 'VAR227'], axis=1)

#X = df_f.drop(columns = ['MRC_ID_DI', 'VAR002', 'VAR003', 'VAR004', 'VAR006', 'VAR007', 'VAR008', 'VAR009', 'VAR011', 'VAR012', 'VAR015',
#                        'VAR016', 'VAR018', 'VAR021', 'VAR022', 'VAR024', 'VAR025', 'VAR027', 'VAR028', 'VAR034', 'VAR036', 'VAR038',
#                        'VAR039', 'VAR046', 'VAR047', 'VAR048', 'VAR049', 'VAR050', 'VAR055', 'VAR056', 'VAR057', 'VAR059', 'VAR062',
#                        'VAR064', 'VAR068', 'VAR069', 'VAR070', 'VAR071', 'VAR075', 'VAR081', 'VAR085', 'VAR087', 'VAR089', 'VAR093', 
#                        'VAR098', 'VAR106', 'VAR107', 'VAR108', 'VAR116', 'VAR118', 'VAR123', 'VAR128', 'VAR130', 'VAR133', 'VAR134',
#                        'VAR139', 'VAR141', 'VAR143', 'VAR144', 'VAR145', 'VAR147', 'VAR148', 'VAR154', 'VAR159', 'VAR162', 'VAR164', 
#                        'VAR170', 'VAR173', 'VAR175', 'VAR182', 'VAR184', 'VAR185', 'VAR188', 'VAR189', 'VAR190', 'VAR192', 'VAR199', 
#                        'VAR200', 'VAR203', 'VAR204', 'VAR214', 'VAR215', 'VAR218', 'VAR227'], axis=1)

y = tf.keras.utils.to_categorical(df_f['MRC_ID_DI'])

In [22]:
num, cat = list(), list()
for i in X.columns:
    if nc.loc[i, 'dType'] == 'numerical':
        num.append(i)
    else:
        if len(df[df[i] == 1]) > len(df)*0.8 or len(df[df[i] == 0]) > len(df)*0.8:
            continue
        else:
            #cat.append(i)
            num.append(i)
X = X[num]

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [24]:
"""
X1_train = X_train[num]
X1_test = X_test[num]

X2_train = X_train[cat]
X2_test = X_test[cat]
"""

'\nX1_train = X_train[num]\nX1_test = X_test[num]\n\nX2_train = X_train[cat]\nX2_test = X_test[cat]\n'

In [25]:
"""
input_1 = tf.keras.Input(dtype = tf.float32, shape = (len(num),))
input_2 = tf.keras.Input(dtype = tf.float32, shape = (len(cat),))

dense_layer_1_1 = tf.keras.layers.Dense(units = 10, activation = tf.nn.relu)(input_1)
dense_layer_1_2 = tf.keras.layers.Dense(units = 10, activation = tf.nn.relu)(dense_layer_1_1)
dense_layer_1_3 = tf.keras.layers.Dense(units = 10, activation = tf.nn.relu)(dense_layer_1_2)
dense_layer_1_4 = tf.keras.layers.Dense(units = 10, activation = tf.nn.relu)(dense_layer_1_3)
dropout_1_5 = tf.keras.layers.Dropout(rate = 0.2)(dense_layer_1_2)


dense_layer_2_1 = tf.keras.layers.Dense(units = 10, activation = tf.nn.relu)(input_2)
dense_layer_2_2 = tf.keras.layers.Dense(units = 10, activation = tf.nn.relu)(dense_layer_2_1)
dense_layer_2_3 = tf.keras.layers.Dense(units = 10, activation = tf.nn.relu)(dense_layer_2_2)
dense_layer_2_4 = tf.keras.layers.Dense(units = 10, activation = tf.nn.relu)(dense_layer_2_3)
dropout_2_5 = tf.keras.layers.Dropout(rate = 0.2)(dense_layer_2_4)

concat_layer = tf.keras.layers.Concatenate()([dropout_1_5, dropout_2_5])


dense_layer_3 = tf.keras.layers.Dense(units = 10, activation = tf.nn.relu)(concat_layer)
dense_layer_4 = tf.keras.layers.Dense(units = 10, activation = tf.nn.relu)(dense_layer_3)
dense_layer_5 = tf.keras.layers.Dense(units = 10, activation = tf.nn.relu)(dense_layer_4)
dense_layer_6 = tf.keras.layers.Dense(units = 10, activation = tf.nn.relu)(dense_layer_5)

output = tf.keras.layers.Dense(units = 2, activation = tf.nn.softmax)(dense_layer_4)
"""

'\ninput_1 = tf.keras.Input(dtype = tf.float32, shape = (len(num),))\ninput_2 = tf.keras.Input(dtype = tf.float32, shape = (len(cat),))\n\ndense_layer_1_1 = tf.keras.layers.Dense(units = 10, activation = tf.nn.relu)(input_1)\ndense_layer_1_2 = tf.keras.layers.Dense(units = 10, activation = tf.nn.relu)(dense_layer_1_1)\ndense_layer_1_3 = tf.keras.layers.Dense(units = 10, activation = tf.nn.relu)(dense_layer_1_2)\ndense_layer_1_4 = tf.keras.layers.Dense(units = 10, activation = tf.nn.relu)(dense_layer_1_3)\ndropout_1_5 = tf.keras.layers.Dropout(rate = 0.2)(dense_layer_1_2)\n\n\ndense_layer_2_1 = tf.keras.layers.Dense(units = 10, activation = tf.nn.relu)(input_2)\ndense_layer_2_2 = tf.keras.layers.Dense(units = 10, activation = tf.nn.relu)(dense_layer_2_1)\ndense_layer_2_3 = tf.keras.layers.Dense(units = 10, activation = tf.nn.relu)(dense_layer_2_2)\ndense_layer_2_4 = tf.keras.layers.Dense(units = 10, activation = tf.nn.relu)(dense_layer_2_3)\ndropout_2_5 = tf.keras.layers.Dropout(rate = 

In [26]:
input_ = tf.keras.Input(dtype = tf.float32, shape = (len(X.columns),))

dense_layer_1_1 = tf.keras.layers.Dense(units = 10, activation = tf.nn.relu)(input_)
dense_layer_1_2 = tf.keras.layers.Dense(units = 10, activation = tf.nn.relu)(dense_layer_1_1)
dense_layer_1_3 = tf.keras.layers.Dense(units = 10, activation = tf.nn.relu)(dense_layer_1_2)
dense_layer_1_4 = tf.keras.layers.Dense(units = 10, activation = tf.nn.relu)(dense_layer_1_3)

dropout_1_5 = tf.keras.layers.Dropout(rate = 0.2)(dense_layer_1_1)

output = tf.keras.layers.Dense(units = 2, activation = tf.nn.softmax)(dropout_1_5)

In [27]:
model = tf.keras.Model(inputs=input_, outputs=output)
#model = tf.keras.Model(inputs=[input_1, input_2], outputs=output)

In [28]:
model.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer=tf.compat.v1.train.AdadeltaOptimizer(learning_rate=0.01), metrics=['acc'])
print(model.summary())

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 204)]             0         
_________________________________________________________________
dense_5 (Dense)              (None, 10)                2050      
_________________________________________________________________
dropout_1 (Dropout)          (None, 10)                0         
_________________________________________________________________
dense_9 (Dense)              (None, 2)                 22        
Total params: 2,072
Trainable params: 2,072
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
#history = model.fit(x=X_train, y=y_train, batch_size=32, epochs=400, verbose=1, validation_split=0.1)
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=100)
history = model.fit(x=X_train, y=y_train, batch_size=32, epochs=2000, verbose=1, validation_split=0.1, callbacks=[callback])
#history = model.fit(x=[X1_train, X2_train], y=y_train, batch_size=32, epochs=2000, verbose=1, validation_split=0.1, callbacks=[callback])

Train on 3116 samples, validate on 347 samples
Epoch 1/2000
3116/3116 [==============================] - 0s 129us/sample - loss: 0.7216 - acc: 0.4544 - val_loss: 0.7132 - val_acc: 0.4640
Epoch 2/2000
3116/3116 [==============================] - 0s 51us/sample - loss: 0.7149 - acc: 0.4689 - val_loss: 0.7070 - val_acc: 0.4755
Epoch 3/2000
3116/3116 [==============================] - 0s 50us/sample - loss: 0.7095 - acc: 0.4894 - val_loss: 0.7012 - val_acc: 0.4841
Epoch 4/2000
3116/3116 [==============================] - 0s 51us/sample - loss: 0.7039 - acc: 0.5055 - val_loss: 0.6956 - val_acc: 0.4986
Epoch 5/2000
3116/3116 [==============================] - 0s 54us/sample - loss: 0.6964 - acc: 0.5209 - val_loss: 0.6903 - val_acc: 0.5245
Epoch 6/2000
3116/3116 [==============================] - 0s 51us/sample - loss: 0.6981 - acc: 0.5157 - val_loss: 0.6854 - val_acc: 0.5476
Epoch 7/2000
3116/3116 [==============================] - 0s 53us/sample - loss: 0.6920 - acc: 0.5427 - val_loss: 0.68

3116/3116 [==============================] - 0s 56us/sample - loss: 0.5611 - acc: 0.7208 - val_loss: 0.5508 - val_acc: 0.7349
Epoch 60/2000
3116/3116 [==============================] - 0s 58us/sample - loss: 0.5587 - acc: 0.7279 - val_loss: 0.5499 - val_acc: 0.7378
Epoch 61/2000
3116/3116 [==============================] - 0s 56us/sample - loss: 0.5554 - acc: 0.7234 - val_loss: 0.5490 - val_acc: 0.7349
Epoch 62/2000
3116/3116 [==============================] - 0s 56us/sample - loss: 0.5592 - acc: 0.7307 - val_loss: 0.5481 - val_acc: 0.7349
Epoch 63/2000
3116/3116 [==============================] - 0s 61us/sample - loss: 0.5605 - acc: 0.7240 - val_loss: 0.5474 - val_acc: 0.7378
Epoch 64/2000
3116/3116 [==============================] - 0s 57us/sample - loss: 0.5527 - acc: 0.7397 - val_loss: 0.5465 - val_acc: 0.7378
Epoch 65/2000
3116/3116 [==============================] - 0s 53us/sample - loss: 0.5583 - acc: 0.7250 - val_loss: 0.5458 - val_acc: 0.7378
Epoch 66/2000
3116/3116 [=========

3116/3116 [==============================] - 0s 67us/sample - loss: 0.5378 - acc: 0.7394 - val_loss: 0.5263 - val_acc: 0.7378
Epoch 118/2000
3116/3116 [==============================] - 0s 64us/sample - loss: 0.5296 - acc: 0.7429 - val_loss: 0.5261 - val_acc: 0.7378
Epoch 119/2000
3116/3116 [==============================] - 0s 59us/sample - loss: 0.5315 - acc: 0.7442 - val_loss: 0.5259 - val_acc: 0.7406
Epoch 120/2000
3116/3116 [==============================] - 0s 59us/sample - loss: 0.5306 - acc: 0.7420 - val_loss: 0.5257 - val_acc: 0.7406
Epoch 121/2000
3116/3116 [==============================] - 0s 59us/sample - loss: 0.5309 - acc: 0.7417 - val_loss: 0.5255 - val_acc: 0.7406
Epoch 122/2000
3116/3116 [==============================] - 0s 60us/sample - loss: 0.5275 - acc: 0.7465 - val_loss: 0.5253 - val_acc: 0.7406
Epoch 123/2000
3116/3116 [==============================] - 0s 59us/sample - loss: 0.5308 - acc: 0.7429 - val_loss: 0.5251 - val_acc: 0.7406
Epoch 124/2000
3116/3116 [==

3116/3116 [==============================] - 0s 58us/sample - loss: 0.5210 - acc: 0.7545 - val_loss: 0.5178 - val_acc: 0.7493
Epoch 176/2000
3116/3116 [==============================] - 0s 54us/sample - loss: 0.5188 - acc: 0.7545 - val_loss: 0.5177 - val_acc: 0.7493
Epoch 177/2000
3116/3116 [==============================] - 0s 55us/sample - loss: 0.5208 - acc: 0.7484 - val_loss: 0.5176 - val_acc: 0.7493
Epoch 178/2000
3116/3116 [==============================] - 0s 61us/sample - loss: 0.5219 - acc: 0.7487 - val_loss: 0.5175 - val_acc: 0.7493
Epoch 179/2000
3116/3116 [==============================] - 0s 60us/sample - loss: 0.5211 - acc: 0.7401 - val_loss: 0.5174 - val_acc: 0.7522
Epoch 180/2000
3116/3116 [==============================] - 0s 60us/sample - loss: 0.5175 - acc: 0.7474 - val_loss: 0.5173 - val_acc: 0.7522
Epoch 181/2000
3116/3116 [==============================] - 0s 62us/sample - loss: 0.5201 - acc: 0.7461 - val_loss: 0.5172 - val_acc: 0.7522
Epoch 182/2000
3116/3116 [==

3116/3116 [==============================] - 0s 62us/sample - loss: 0.5079 - acc: 0.7529 - val_loss: 0.5122 - val_acc: 0.7522
Epoch 234/2000
3116/3116 [==============================] - 0s 62us/sample - loss: 0.5091 - acc: 0.7526 - val_loss: 0.5122 - val_acc: 0.7522
Epoch 235/2000
3116/3116 [==============================] - 0s 66us/sample - loss: 0.5104 - acc: 0.7506 - val_loss: 0.5121 - val_acc: 0.7522
Epoch 236/2000
3116/3116 [==============================] - 0s 59us/sample - loss: 0.5120 - acc: 0.7497 - val_loss: 0.5120 - val_acc: 0.7522
Epoch 237/2000
3116/3116 [==============================] - 0s 60us/sample - loss: 0.5119 - acc: 0.7522 - val_loss: 0.5119 - val_acc: 0.7522
Epoch 238/2000
3116/3116 [==============================] - 0s 61us/sample - loss: 0.5126 - acc: 0.7539 - val_loss: 0.5118 - val_acc: 0.7522
Epoch 239/2000
3116/3116 [==============================] - 0s 59us/sample - loss: 0.5112 - acc: 0.7542 - val_loss: 0.5117 - val_acc: 0.7522
Epoch 240/2000
3116/3116 [==

...

Epoch 1624/2000
3116/3116 [==============================] - 0s 54us/sample - loss: 0.4430 - acc: 0.7940 - val_loss: 0.4633 - val_acc: 0.7925
Epoch 1625/2000
3116/3116 [==============================] - 0s 54us/sample - loss: 0.4400 - acc: 0.7969 - val_loss: 0.4633 - val_acc: 0.7925
Epoch 1626/2000
3116/3116 [==============================] - 0s 54us/sample - loss: 0.4398 - acc: 0.7953 - val_loss: 0.4633 - val_acc: 0.7925
Epoch 1627/2000
3116/3116 [==============================] - 0s 59us/sample - loss: 0.4434 - acc: 0.7936 - val_loss: 0.4633 - val_acc: 0.7925
Epoch 1628/2000
3116/3116 [==============================] - 0s 58us/sample - loss: 0.4398 - acc: 0.7978 - val_loss: 0.4633 - val_acc: 0.7925
Epoch 1629/2000
3116/3116 [==============================] - 0s 53us/sample - loss: 0.4380 - acc: 0.8001 - val_loss: 0.4633 - val_acc: 0.7925
Epoch 1630/2000
3116/3116 [==============================] - 0s 53us/sample - loss: 0.4411 - acc: 0.7959 - val_loss: 0.4633 - val_acc: 0.7925
Epoch 

Epoch 1682/2000
3116/3116 [==============================] - 0s 53us/sample - loss: 0.4358 - acc: 0.7991 - val_loss: 0.4625 - val_acc: 0.7925
Epoch 1683/2000
3116/3116 [==============================] - 0s 53us/sample - loss: 0.4393 - acc: 0.7988 - val_loss: 0.4625 - val_acc: 0.7925
Epoch 1684/2000
3116/3116 [==============================] - 0s 54us/sample - loss: 0.4400 - acc: 0.7959 - val_loss: 0.4625 - val_acc: 0.7925
Epoch 1685/2000
3116/3116 [==============================] - 0s 53us/sample - loss: 0.4405 - acc: 0.7924 - val_loss: 0.4625 - val_acc: 0.7925
Epoch 1686/2000
3116/3116 [==============================] - 0s 59us/sample - loss: 0.4427 - acc: 0.8013 - val_loss: 0.4624 - val_acc: 0.7925
Epoch 1687/2000
3116/3116 [==============================] - 0s 54us/sample - loss: 0.4338 - acc: 0.7988 - val_loss: 0.4624 - val_acc: 0.7925
Epoch 1688/2000
3116/3116 [==============================] - 0s 54us/sample - loss: 0.4441 - acc: 0.7962 - val_loss: 0.4624 - val_acc: 0.7925
Epoch 

Epoch 1740/2000
3116/3116 [==============================] - 0s 61us/sample - loss: 0.4416 - acc: 0.7965 - val_loss: 0.4619 - val_acc: 0.7925
Epoch 1741/2000
3116/3116 [==============================] - 0s 55us/sample - loss: 0.4415 - acc: 0.7988 - val_loss: 0.4619 - val_acc: 0.7925
Epoch 1742/2000
3116/3116 [==============================] - 0s 59us/sample - loss: 0.4375 - acc: 0.7978 - val_loss: 0.4619 - val_acc: 0.7925
Epoch 1743/2000
3116/3116 [==============================] - 0s 55us/sample - loss: 0.4365 - acc: 0.7985 - val_loss: 0.4618 - val_acc: 0.7925
Epoch 1744/2000
3116/3116 [==============================] - 0s 53us/sample - loss: 0.4389 - acc: 0.7956 - val_loss: 0.4619 - val_acc: 0.7925
Epoch 1745/2000
3116/3116 [==============================] - 0s 54us/sample - loss: 0.4405 - acc: 0.8004 - val_loss: 0.4619 - val_acc: 0.7925
Epoch 1746/2000
3116/3116 [==============================] - 0s 54us/sample - loss: 0.4421 - acc: 0.7940 - val_loss: 0.4618 - val_acc: 0.7925
Epoch 

Epoch 1798/2000
3116/3116 [==============================] - 0s 62us/sample - loss: 0.4377 - acc: 0.8004 - val_loss: 0.4612 - val_acc: 0.7925
Epoch 1799/2000
3116/3116 [==============================] - 0s 57us/sample - loss: 0.4384 - acc: 0.7953 - val_loss: 0.4612 - val_acc: 0.7925
Epoch 1800/2000
3116/3116 [==============================] - 0s 53us/sample - loss: 0.4383 - acc: 0.8004 - val_loss: 0.4612 - val_acc: 0.7925
Epoch 1801/2000
3116/3116 [==============================] - 0s 67us/sample - loss: 0.4368 - acc: 0.8017 - val_loss: 0.4612 - val_acc: 0.7925
Epoch 1802/2000
3116/3116 [==============================] - 0s 56us/sample - loss: 0.4396 - acc: 0.7972 - val_loss: 0.4612 - val_acc: 0.7925
Epoch 1803/2000
3116/3116 [==============================] - 0s 57us/sample - loss: 0.4390 - acc: 0.7946 - val_loss: 0.4612 - val_acc: 0.7925
Epoch 1804/2000
3116/3116 [==============================] - 0s 57us/sample - loss: 0.4414 - acc: 0.7981 - val_loss: 0.4611 - val_acc: 0.7925
Epoch 

Epoch 1856/2000
3116/3116 [==============================] - 0s 54us/sample - loss: 0.4390 - acc: 0.7994 - val_loss: 0.4605 - val_acc: 0.7954
Epoch 1857/2000
3116/3116 [==============================] - 0s 58us/sample - loss: 0.4368 - acc: 0.7959 - val_loss: 0.4605 - val_acc: 0.7954
Epoch 1858/2000
3116/3116 [==============================] - 0s 59us/sample - loss: 0.4448 - acc: 0.7946 - val_loss: 0.4605 - val_acc: 0.7954
Epoch 1859/2000
3116/3116 [==============================] - 0s 55us/sample - loss: 0.4376 - acc: 0.8020 - val_loss: 0.4605 - val_acc: 0.7954
Epoch 1860/2000
3116/3116 [==============================] - 0s 54us/sample - loss: 0.4355 - acc: 0.7949 - val_loss: 0.4605 - val_acc: 0.7954
Epoch 1861/2000
3116/3116 [==============================] - 0s 58us/sample - loss: 0.4364 - acc: 0.7997 - val_loss: 0.4605 - val_acc: 0.7954
Epoch 1862/2000
3116/3116 [==============================] - 0s 55us/sample - loss: 0.4358 - acc: 0.7985 - val_loss: 0.4605 - val_acc: 0.7954
Epoch 

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])

plt.title('model accuracy')
plt.ylabel('acc')
plt.xlabel('epoch')
plt.legend(['train','test'], loc='upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])

plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train','test'], loc='upper left')
plt.show()

In [ ]:
score = model.evaluate(x=X_test, y=y_test, verbose=1)
#score = model.evaluate(x=[X1_test, X2_test], y=y_test, verbose=1)

print("Test Score:", score[0])
print("Test ACC:", score[1])

In [ ]:
learning_rate = 0.00005
training_epochs = 400
batch_size = 32
num_models = 10

tf.compat.v1.disable_eager_execution()

class Model:

    def __init__(self, sess, name):
        self.sess = sess
        self.name = name
        self._build_net()

    def _build_net(self):
        with tf.compat.v1.variable_scope(self.name):
            
            self.training = tf.keras.backend.placeholder(dtype = tf.bool)

            self.X = tf.keras.backend.placeholder(dtype = tf.float32, shape=(None,len(X_train.columns)))            
            self.Y = tf.keras.backend.placeholder(dtype = tf.float32, shape=(None,2))

            dense1 = tf.compat.v1.layers.dense(inputs=self.X, units = 10, activation=tf.nn.relu)
            dropout1 = tf.compat.v1.layers.dropout(inputs=dense1, rate=0.2, training=self.training)
            
            self.logits = tf.keras.layers.Dense(units = 2, activation = tf.nn.softmax)(dropout1)

        self.cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=self.logits, labels=self.Y))
        self.optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=learning_rate).minimize(self.cost)

        correct_prediction = tf.equal(tf.argmax(self.logits, 1), tf.argmax(self.Y, 1))
        self.accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    def predict(self, x_test, training=False):
        return self.sess.run(self.logits,
                             feed_dict={self.X: x_test, self.training: training})

    def get_accuracy(self, x_test, y_test, training=False):
        return self.sess.run(self.accuracy,
                             feed_dict={self.X: x_test,
                                        self.Y: y_test, self.training: training})

    def train(self, x_data, y_data, training=True):
        return self.sess.run([self.cost, self.optimizer], feed_dict={
            self.X: x_data, self.Y: y_data, self.training: training})

    
sess = tf.compat.v1.Session()


models = []
for m in range(num_models):
    models.append(Model(sess, "model" + str(m)))

sess.run(tf.compat.v1.global_variables_initializer())

print('Learning Started!')


for epoch in range(training_epochs):
    avg_cost_list = np.zeros(len(models))
    total_batch = int(len(X_train)/ batch_size)
    for i in range(total_batch):
        batch_xs, batch_ys = X_train[i * batch_size:(i+1) * batch_size], y_train[i * batch_size:(i+1) * batch_size]
        for m_idx, m in enumerate(models):
            c, _ = m.train(batch_xs, batch_ys)
            avg_cost_list[m_idx] += c / total_batch
    print('Epoch:', '%04d' % (epoch + 1), 'cost =', avg_cost_list)

print('Learning Finished!')


test_size = len(y_test)
predictions = np.zeros([test_size, 2])
for m_idx, m in enumerate(models):
    print(m_idx, 'Accuracy:', m.get_accuracy(
        X_test, y_test))
    p = m.predict(X_test) 
    predictions += p

ensemble_correct_prediction = tf.equal(tf.argmax(predictions, 1), tf.argmax(y_test, 1))
ensemble_accuracy = tf.reduce_mean(tf.cast(ensemble_correct_prediction,  tf.float32))
print('Ensemble accuracy:', sess.run(ensemble_accuracy))